In [ ]:
import networkx as nx
from collections import defaultdict
from itertools import combinations
import random
import math
from dwave.system import DWaveSampler, EmbeddingComposite, DWaveCliqueSampler, FixedEmbeddingComposite
import neal
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl
from minorminer import find_embedding
import greedy

## Create Work Distribution and Embeddings

In [ ]:

data = np.genfromtxt('../wp3')  #Load dataset
wp=data[:,2]
nproc=2 # Number of processors to partition

In [ ]:
w00=wp[0:60] #Adjust size of problem as needed

In [ ]:
# Create Embeddings 
final=[]
final.append(w00)

for kk in range(nproc-1):
    w=final[kk] # Store initial WP
    h = {} # Empty H 
    J={}   # Create J
    for i in range(len(w)):
     for j in range(i+1,len(w)): 
       J[(i,j)]= w[i]*w[j] #Off diagonal terms only 

# Combine to form Q
Q = {**h, **J}


# Target D-Wave hardware graph (e.g., a Chimera graph)
sampler = DWaveSampler()  # Initializes a D-Wave Sampler
target_graph = sampler.edgelist

# Generate the embedding
embedding1 = find_embedding(Q.keys(), target_graph)
embedding2 = find_embedding(Q.keys(), target_graph)
embedding3 = find_embedding(Q.keys(), target_graph)
embedding4 = find_embedding(Q.keys(), target_graph)
embedding5 = find_embedding(Q.keys(), target_graph)


In [ ]:
#=============================================Save embeddings 
#    'embedding1': embedding1,
#    'embedding2': embedding2,
#    'embedding3': embedding3,
#    'embedding4': embedding4,
#    'embedding5': embedding5,
#}
#import pickle
#with open('embeddings.pkl', 'wb') as f:
#    pickle.dump(embeddings, f)
    
#==============================================Load embeddings
#with open('../embeddings.pkl', 'rb') as f:
#    loaded_embeddings = pickle.load(f)


## Impact of chain strengths

In [ ]:
#Run 5 repetitions using embeddings for each chain strength
sampler=DWaveSampler(token='')   #Insert token


cbf_values = {} #Chainbreak values
err_values = {} #Solution Quality

cs_norm=max(w00)
for cs in [100,500,1000,1500, 2000,3000,6000,10000,20000]:   #Edit Range If Needed
     for embedding_name, embedding in [('embedding1', embedding1),('embedding2', embedding2),
                                       ('embedding3', embedding3),('embedding4', embedding4),
                                       ('embedding5', embedding5)]: #For the prepared embeddings
        
        final=[]
        final.append(w00)
        
        for kk in range(nproc-1):  #Assuming 2 procs, this loops only triggers once here. In general this is not necessarily  true.
            w=final[kk] # Store initial WP
            h = {} 
            J={}   
            for i in range(len(w)):
             for j in range(i+1,len(w)): 
               J[(i,j)]= w[i]*w[j] 
        
            sampler_embedded = FixedEmbeddingComposite(sampler, embedding)
            sampleset = sampler_embedded.sample_ising(h, J,
                                    num_reads = 1000,
                                    label='load',
                                    chain_strength=cs*cs_norm)

            wlist_1=[]      #list of weights
            wlist_2=[]

            for part, proc in sampleset.first.sample.items(): #Isolate the best solution and create 2 sets (i.e. one for each processor)
                if proc == -1 : 
                    wlist_1.append(w[part])
                else :
                    wlist_2.append(w[part])


            final.append(wlist_1)  # Add new split levels to final (In general can have nested levels for multiple processors) 
            final.append(wlist_2)
            #Store CBF and Error 
            cbf=sampleset.first.chain_break_fraction                       #calculate cbf
            cbf_values.setdefault((cs, embedding_name), []).append(cbf)    #store cbf
            err=abs(sum(final[1])-sum(final[2]))/(0.5*(sum(final[1])+sum(final[2]))) #calculate err
            err_values.setdefault((cs, embedding_name), []).append(err)              #store err
        print("Final Output")
        for i in range(len(final)):
            print("Level=",i,sum(final[i]))

In [ ]:
#Snippet to aggregate then average cbf values across all embeddings for each cs
averages_cbf = {}
cbf_bars = {} 

# First, aggregate all kk1 values for each chain strength (cs)
aggregated_values = {}
for (cs, embedding_name), values in cbf_values.items():
    aggregated_values.setdefault(cs, []).extend(values)

# Now, compute the averages
for cs, values in aggregated_values.items():
    avg_kk1 = sum(values) / len(values)
    averages_cbf[cs] = avg_kk1
#Compute error bars
    variance = sum([(x - avg_kk1) ** 2 for x in values]) / len(values)
    std_dev = math.sqrt(variance)
    cbf_bars[cs] = std_dev

In [ ]:
#Snippet to aagregate then average ERR values across all embeddings for each cs
averages_err = {}
err_bars = {} 
# First, aggregate all kk1 values for each chain strength (cs)
aggregated_values = {}
for (cs, embedding_name), values in err_values.items():
    aggregated_values.setdefault(cs, []).extend(values)

# Now, compute the averages
for cs, values in aggregated_values.items():
    avg_kk1 = sum(values) / len(values)
    averages_err[cs] = avg_kk1
#Compute error bars
    variance = sum([(x - avg_kk1) ** 2 for x in values]) / len(values)
    std_dev = math.sqrt(variance)
    err_bars[cs] = std_dev

In [ ]:
#plot of CBF against CS
chain_strengths = list(averages_cbf.keys())
mean_values = list(averages_cbf.values())
sd_values = list(cbf_bars.values())

plt.errorbar(chain_strengths, mean_values, yerr=sd_values, fmt='o', capsize=5, label='Mean with SD')
plt.xlabel("Chain Strength",fontweight='bold')
plt.ylabel("Chain Break Fraction",fontweight='bold')
plt.tight_layout()
#plt.savefig("cbf_cs.pdf")

In [ ]:
#Plt of error agaisnt cs with inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

chain_strengths = list(averages_err.keys())
mean_values = list(averages_err.values())
sd_values = list(err_bars.values())

fig, ax = plt.subplots()

# Main plot
ax.errorbar(chain_strengths, mean_values, yerr=sd_values, fmt='o', capsize=5, label='Mean with SD')
plt.xlabel("Chain Strength", fontweight='bold')
plt.ylabel("Solution Quality",fontweight='bold')

# Define the inset axis
axins = inset_axes(ax, width='30%', height='30%', loc='upper right',
                  bbox_to_anchor=(-0.01, 0.01, 1, 1), bbox_transform=ax.transAxes)

# Plot the last 3 points on the inset axis
x_last3 = chain_strengths[-3:]
y_last3 = mean_values[-3:]
axins.errorbar(chain_strengths[-3:], mean_values[-3:], yerr=sd_values[-3:], fmt='o', capsize=5, label='Mean with SD')
axins.set_xlim(min(x_last3)-500, max(x_last3)+500)  # set x limits for clarity
axins.set_ylim(min(y_last3)-0.0003, max(y_last3)+0.0005)  # set y limits for clarity



plt.tight_layout()
#plt.savefig("err_cs2.pdf")


In [ ]:
#Plt of error agaisnt cs with inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

chain_strengths = list(averages_cbf.keys())
mean_values = list(averages_cbf.values())
sd_values = list(cbf_bars.values())

fig, ax = plt.subplots()

# Main plot
ax.errorbar(chain_strengths, mean_values, yerr=sd_values, fmt='o', capsize=5, label='Mean with SD')
plt.xlabel("Chain Strength", fontweight='bold')
plt.ylabel("Chain Break Fraction",fontweight='bold')

# Define the inset axis
axins = inset_axes(ax, width='30%', height='30%', loc='upper right',
                  bbox_to_anchor=(-0.01, 0.01, 1, 1), bbox_transform=ax.transAxes)

# Plot the last 3 points on the inset axis
x_last3 = chain_strengths[-3:]
y_last3 = mean_values[-3:]
axins.errorbar(chain_strengths[-3:], mean_values[-3:], yerr=sd_values[-3:], fmt='o', capsize=5, label='Mean with SD')
axins.set_xlim(min(x_last3)-500, max(x_last3)+500)  # set x limits for clarity
axins.set_ylim(min(y_last3)-0.02, max(y_last3)+0.08)  # set y limits for clarity



plt.tight_layout()
#plt.savefig("cbf_cs2.pdf")


In [ ]:
#Chainbreaks vs problem size, computed using the same h/J matrices but default chain strength
data_column_names = ['x', 'y']
error_column_names = ['x','y_error']

# Read the CSV files, assuming no headers
# Replace 'data.csv' and 'error_bars.csv' with your actual file names
data_df = pd.read_csv('cbf.csv', header=None, names=data_column_names)
error_df = pd.read_csv('cbf_err.csv', header=None, names=error_column_names)

error= abs(data_df['y'] - error_df['y_error'])
#data_df['y']
#error_df['y_error']
fig=plt.figure(figsize=(8, 6), dpi=100)
plt.scatter(data_df['x'],data_df['y'],marker='o',s=70)
plt.errorbar(data_df['x'],data_df['y'],yerr=error,linestyle="none", capsize=3)
plt.xlabel('Problem Size')
plt.ylabel('Chain Break Fraction')
#plt.savefig("cbf.pdf", bbox_inches="tight")

## Number of Anneals 

In [ ]:
#Finding effect of anneals - Loop this code as required over range of desired number of anneals
#For each configuration, it will store results in solq_final

#Run recursive partition 5 times 
solq_final=[]
for tt in range(5):
    final=[]
    final.append(w00)

    final2=[]

    for kk in range(nproc-1):
        w=final[kk] # Store initial WP
        h = {} # Empty H 
        J={}   # Create J
        for i in range(len(w)):
         for j in range(i+1,len(w)): 
           J[(i,j)]= w[i]*w[j] 


# Define the sampler
#========================================================
        sampler = EmbeddingComposite(DWaveSampler())
    #sampler = neal.SimulatedAnnealingSampler()
    #sampler = greedy.SteepestDescentSolver()
# Run the problem on the sampler and print the results
        sampleset = sampler.sample_ising(h, J,
                                     num_reads = 25,
                                     label='Load')

    #print(sampleset.first.sample)
        plist_1=[]      #list of particles 
        plist_2=[]
        wlist_1=[]      #list of weights
        wlist_2=[]

        for part, proc in sampleset.first.sample.items():
        #print(part,":",proc)
            if proc == -1 : 
                plist_1.append(part)
                wlist_1.append(w[part])
            else :
                plist_2.append(part)
                wlist_2.append(w[part])


        final.append(wlist_1)  # Add new split levels to final 
        final.append(wlist_2)

    print("Final Output")
    for i in range(len(final)):
        print("Level=",i,sum(final[i]))
    #print("Level=",i,sum(final[i]), file=open('test_gr.dat', 'a'))
    print(plist_1)
    print(plist_2)
    df=sampleset.to_pandas_dataframe()
    out="b25_"+str(tt+1)+".dat"
    solq=abs(sum(final[1])-sum(final[2]))/(0.5*(sum(final[1])+sum(final[2])))
    solq_final.append(solq)

In [ ]:
# Combine precomputed results
solq_final_vars = [solq_final, solq2_final, solq3_final, solq4_final, solq5_final, solq6_final, solq7_final, solq8_final]

# Calculate mean and std
c1 = [np.mean(solq) for solq in solq_final_vars]
s1 = [np.std(solq) for solq in solq_final_vars]

# Define x-axis values i.e. number of anneals
x = [25, 50, 100, 200, 500, 1000, 1500, 2000]

plt.figure(figsize=(8, 5))
plt.scatter(x, c1, label="Mean Solution Quality", color='blue', zorder=5)
plt.errorbar(x, c1, yerr=s1, linestyle="none", capsize=3, color='blue', zorder=4)
plt.ylabel("Solution Quality")
plt.xlabel("Number of Anneals")
plt.title("Effect of Number of Anneals on Solution Quality")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)

# Save plot to a file
#plt.savefig("number_anneals.pdf")
plt.show()

## Likelihood of good solution

In [ ]:
#Round Robin Results

def round_robin_partition2(numbers):
    num_partitions = 2
    partitions = [[] for _ in range(num_partitions)]  # Create empty partitions

    for i, num in enumerate(sorted(numbers)):
        partition_index = i % num_partitions  # Determine the partition index using modulo operation
        partitions[partition_index].append(num)  # Add the number to the corresponding partition

    return partitions
data = np.genfromtxt('../wp2')  ##Choose a dataset i.e. wp2 or wp3
numbers=data[:,2]
numbers=numbers[0:60] #Truncate problem size

#Recursively apply RR
result = round_robin_partition2(numbers)


print("Partition 1:", sum(result[0]))
print("Partition 2:", sum(result[1]))

zz=(sum(result[1])-sum(result[0]))/(sum(result[0])+sum(result[1])) #needed for plot


In [ ]:
#Original Recursive Partition for single run and single problem size

data = np.genfromtxt('../wp2')  #Choose a dataset i.e. wp2 or wp3
w00=data[:,2]
w00=w00[0:60] #Truncate problem size

final=[]
final.append(w00)
for kk in range(nproc-1):
    w=final[kk] # Store initial WP
    h = {} # Empty H 
    J={}   # Create J
    for i in range(len(w)):
     for j in range(i+1,len(w)): 
       J[(i,j)]= w[i]*w[j] #


# Define the sampler that will be used to run the problem
#========================================================
    sampler = EmbeddingComposite(DWaveSampler())

    #sampler = greedy.SteepestDescentSolver()
# Run the problem on the sampler and print the results
#    sampleset = sampler.sample_ising(h, J,
#                                 num_reads = 100,
#                                 label='Load')
    sampleset = sampler.sample_ising(h, J,
                                 num_reads = 100,
                                 label='load',chain_strength=100000)  #Adjust chainstrenght if needed

    #print(sampleset.first.sample)
    plist_1=[]      #list of particles 
    plist_2=[]
    wlist_1=[]      #list of weights
    wlist_2=[]

    for part, proc in sampleset.first.sample.items():
        #print(part,":",proc)
        if proc == -1 : 
            plist_1.append(part)
            wlist_1.append(w[part])
        else :
            plist_2.append(part)
            wlist_2.append(w[part])


    final.append(wlist_1)  # Add new split levels to final 
    final.append(wlist_2)

print("Final Output")
for i in range(len(final)):
    print("Level=",i,sum(final[i]))
    #print("Level=",i,sum(final[i]), file=open('test_gr.dat', 'a'))
print(plist_1)
print(plist_2)

In [ ]:
c5=sampleset.to_pandas_dataframe()
sol_std=[]
sol_org=c5.sort_values(by="energy",ascending=True)
for j in range(0,len(sol_org)):      # Looping over solutions
    
    sol=sol_org.iloc[j]
    sol_t1=[]
    sol_t2=[]
    sol_final=[]

    for i in range(0,len(w00)):   #Looping over patches
        if (sol[i] < 0):
            sol_t1.append(w[i])
        else:
            sol_t2.append(w[i])
    sol_final=[]
    sol_final.append(sum(sol_t1))
    sol_final.append(sum(sol_t2))
    #sol_std.append(np.std(sol_final))  
    sol_std.append(abs(sol_final[0]-sol_final[1])/sum(sol_final))
#sol_std=abs(sol_final[0]-sol_final[1])/sum(sol_final)


In [ ]:
plt.figure()
plt.scatter(sol_org["energy"], sol_std,marker="o",label="QA")
plt.xlabel("Energy")
plt.ylabel("Solution Imbalance")
#plt.title("Solution Quality for all samples of Case C5")b
plt.axhline(y=0,color="r",linestyle="--",label="SD") #SD case-previous code block yields perfect solution i.e. y=o
plt.axhline(y=zz,color="k",linestyle="--",label="RR")
plt.legend()
#plt.savefig("solution_qaulity.pdf")

## Supplementary Figures

In [ ]:
#Round Robin Results 

def round_robin_partition2(numbers):
    num_partitions = 2
    partitions = [[] for _ in range(num_partitions)]  # Create empty partitions

    for i, num in enumerate(sorted(numbers)):
        partition_index = i % num_partitions  # Determine the partition index using modulo operation
        partitions[partition_index].append(num)  # Add the number to the corresponding partition

    return partitions
data = np.genfromtxt('../wp2')  #Load dataset
numbers=data[:,2]
numbers=numbers[0:100]

#Recursively apply RR 
result = round_robin_partition2(numbers)

result2a=round_robin_partition2(result[0])
result2b=round_robin_partition2(result[1])

result3a=round_robin_partition2(result2a[0])
result3b=round_robin_partition2(result2a[1])
result3c=round_robin_partition2(result2b[0])
result3d=round_robin_partition2(result2b[1])

result4a=round_robin_partition2(result3a[0])
result4b=round_robin_partition2(result3a[1])
result4c=round_robin_partition2(result3b[0])
result4d=round_robin_partition2(result3b[1])
result4e=round_robin_partition2(result3c[0])
result4f=round_robin_partition2(result3c[1])
result4g=round_robin_partition2(result3d[0])
result4h=round_robin_partition2(result3d[1])
print("Partition 1:", sum(result[0]))
print("Partition 2:", sum(result[1]))



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load Data from Files
read1 = pd.read_csv("../../test_qa.dat", delim_whitespace=True, header=None, names=["a", "b", "c"], comment="#")
read2 = pd.read_csv("../../test_sa.dat", delim_whitespace=True, header=None, names=["a", "b", "c"], comment="#")
read3 = pd.read_csv("../../test_gr.dat", delim_whitespace=True, header=None, names=["a", "b", "c"], comment="#")

# Extract the 'c' column data for each dataset
dd, dd2, dd3 = read1["c"], read2["c"], read3["c"]

# Define x-axis values for different processor levels
x1, x2, x3, x4 = [1, 2], [1, 2, 3, 4], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

# Define split levels for each dataset
#This is level 0 i.e. Total Work
split_t, split2_t, split3_t = dd[0], dd2[0], dd3[0]
#This is level 1 i.e. 2 procs
split_1, split2_1, split3_1 = (dd[1:3] - split_t / 2) / split_t * 100, (dd2[1:3] - split2_t / 2) / split2_t * 100, (dd3[1:3] - split3_t / 2) / split3_t * 100
#This is level 2 i.e. 4 procs
split_2, split2_2, split3_2 = (dd[3:7] - split_t / 4) / split_t * 100, (dd2[3:7] - split2_t / 4) / split2_t * 100, (dd3[3:7] - split3_t / 4) / split3_t * 100
#This is level 3 i.e. 8 procs
split_3, split2_3, split3_3 = (dd[7:15] - split_t / 8) / split_t * 100, (dd2[7:15] - split2_t / 8) / split2_t * 100, (dd3[7:15] - split3_t / 8) / split3_t * 100
#This is level 4 i.e. 16 procs
split_4, split2_4, split3_4 = (dd[15:31] - split_t / 16) / split_t * 100, (dd2[15:31] - split2_t / 16) / split2_t * 100, (dd3[15:31] - split3_t / 16) / split3_t * 100

# Calculate sum ranges (ss1 - ss4) for each partition level
ss1 = [(sum(result[0]) - split_t / 2) / split_t * 100, (sum(result[1]) - split_t / 2) / split_t * 100]
ss2 = [(sum(result2a[0]) - split_t / 4) / split_t * 100, (sum(result2a[1]) - split_t / 4) / split_t * 100, (sum(result2b[0]) - split_t / 4) / split_t * 100, (sum(result2b[1]) - split_t / 4) / split_t * 100]
ss3 = [(sum(result3a[0]) - split_t / 8) / split_t * 100, (sum(result3a[1]) - split_t / 8) / split_t * 100, (sum(result3b[0]) - split_t / 8) / split_t * 100, (sum(result3b[1]) - split_t / 8) / split_t * 100,
       (sum(result3c[0]) - split_t / 8) / split_t * 100, (sum(result3c[1]) - split_t / 8) / split_t * 100, (sum(result3d[0]) - split_t / 8) / split_t * 100, (sum(result3d[1]) - split_t / 8) / split_t * 100]
ss4 = [(sum(result4a[0]) - split_t / 16) / split_t * 100, (sum(result4a[1]) - split_t / 16) / split_t * 100, (sum(result4b[0]) - split_t / 16) / split_t * 100, (sum(result4b[1]) - split_t / 16) / split_t * 100,
       (sum(result4c[0]) - split_t / 16) / split_t * 100, (sum(result4c[1]) - split_t / 16) / split_t * 100, (sum(result4d[0]) - split_t / 16) / split_t * 100, (sum(result4d[1]) - split_t / 16) / split_t * 100,
       (sum(result4e[0]) - split_t / 16) / split_t * 100, (sum(result4e[1]) - split_t / 16) / split_t * 100, (sum(result4f[0]) - split_t / 16) / split_t * 100, (sum(result4f[1]) - split_t / 16) / split_t * 100,
       (sum(result4g[0]) - split_t / 16) / split_t * 100, (sum(result4g[1]) - split_t / 16) / split_t * 100, (sum(result4h[0]) - split_t / 16) / split_t * 100, (sum(result4h[1]) - split_t / 16) / split_t * 100]

# Plot scatter for each partition level
plt.rcParams.update({'font.size': 16})
plt.tick_params(axis='both', which='major', labelsize=16)

# Scatter plots for each level
for i, (x, splits, ss) in enumerate(zip([x1, x2, x3, x4], [(split_1, split2_1, split3_1), (split_2, split2_2, split3_2), (split_3, split2_3, split3_3), (split_4, split2_4, split3_4)], [ss1, ss2, ss3, ss4])):
    fig = plt.figure(figsize=(8, 6), dpi=100)
    plt.scatter(x, splits[0], marker="+", label="QA", s=100)
    plt.scatter(x, splits[1], marker=".", label="SA", s=100)
    plt.scatter(x, splits[2], marker="x", label="SD", s=100)
    plt.scatter(x, ss, marker="1", label="RR", s=100)
    plt.axhline(0, color="k", linestyle="--")
    plt.xlabel("Proc ID")
    plt.xticks(x)
    plt.ylabel("Imbalance")
    plt.legend()
    # plt.savefig(f"split{i+1}v4.pdf", bbox_inches="tight")

# Calculate and plot the range of imbalance across methods
range_data = {'QA': [], 'SA': [], 'SD': [], 'RR': []}
method_data = {
    'QA': [split_1, split_2, split_3, split_4],
    'SA': [split2_1, split2_2, split2_3, split2_4],
    'SD': [split3_1, split3_2, split3_3, split3_4],
    'RR': [ss1, ss2, ss3, ss4]
}

for label, splits in method_data.items():
    range_data[label] = [max(kk) - min(kk) for kk in splits]

# Plot range comparison
xticks = ["2", "4", "8", "16"]
plt.figure(figsize=(8, 6))
plt.plot(xticks, range_data['QA'], marker="+", label="QA")
plt.plot(xticks, range_data['SA'], marker=".", label="SA")
plt.plot(xticks, range_data['SD'], marker="x", label="SD")
plt.plot(xticks, range_data['RR'], marker="1", label="RR")
plt.xlabel("Processors")
plt.ylabel("Range")

plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
# plt.savefig("range.png", bbox_inches="tight")
plt.show()
